In [1]:
import pandas as pd
import numpy as np

In [2]:
import matplotlib.pyplot as plt

In /Users/djbetts/opt/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /Users/djbetts/opt/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /Users/djbetts/opt/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /Users/djbetts/opt/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /Users/djbetts/opt/anaconda3/lib/

In [3]:
from datetime import datetime

In [4]:
#Current SQL query
"""
SELECT 
geo__country, 
geo__city,
event_params__video_title,
event_params__videotitle,
event_name,
COUNT(event_name) AS times_completed,
FROM `jfp-data-warehouse.production_firebase.firebase_event` 
WHERE geo__country in ('Russia', 'China', 'Turkey', 'Thailand', 'Iran', 'Vietnam', 'Egypt', 'Japan', 'Bangladesh', 'Pakistan', 'Indonesia', 'India') AND 
event_params__video_title IS NOT NULL AND
event_name IN ('videostarts',  'videoplay',  'a_media_progress10',  'a_media_progress25',  'a_media_progress50',  'a_media_progress75',  'a_media_progress90') AND 
DATE(event_date_partition) 
BETWEEN  "2021-01-01" AND "2021-03-31" 
GROUP BY 
geo__country, 
geo__city,
event_params__videotitle, 
event_params__video_title,
event_name
ORDER BY 
geo__country,
event_params__video_title,
times_completed DESC 
LIMIT 15000
""";

In [5]:
data = pd.read_csv('/Users/djbetts/Desktop/jf_project/data/recommender_base.csv')

In [6]:
df = data.copy()
df.drop(['geo__city', 'times_completed'], axis=1, inplace=True)

In [7]:
df.columns

Index(['geo__country', 'event_params__video_title', 'event_params__videotitle',
       'event_name'],
      dtype='object')

In [8]:
df['video_title'] = df['event_params__videotitle'].fillna(df['event_params__video_title'])

In [9]:
df.drop(['event_params__videotitle','event_params__video_title'], axis=1, inplace=True)

In [10]:
(df.geo__country.value_counts())

India         8644
Egypt         2467
Bangladesh    1953
China         1936
Name: geo__country, dtype: int64

In [11]:
df.event_name.value_counts()

videoplay             2741
videostarts           2670
a_media_progress10    2230
a_media_progress25    2093
a_media_progress50    1908
a_media_progress75    1731
a_media_progress90    1627
Name: event_name, dtype: int64

In [12]:
df.video_title.value_counts()

JESUS                       2988
Childhood of Jesus           893
Baptism of Jesus by John     876
Birth of Jesus               856
Abraham                      345
                            ... 
使徒行傳                           1
Jesus Gathers Disciples        1
#फालिंगप्लेट्स                 1
The Wilson Kipsang Story       1
與耶穌的七日：恩典                      1
Name: video_title, Length: 406, dtype: int64

In [13]:
df_90 = df[df['event_name'] == 'a_media_progress90']

In [14]:
df_90.video_title.unique()

array(['#FallingPlates', '1. Jesus, Our Loving Pursuer',
       '7. Jesus Our Living Water', 'Angels at the Tomb', 'B Me',
       'Baptism of Jesus by John', 'Beatitudes', 'Birth of Jesus',
       'Blessed are those Who Hear and Obey', 'Breathe',
       'Burial of Jesus', 'Can the Bible Be Trusted?',
       'Childhood of Jesus', 'Creation', 'Crucified Convicts',
       'Death of Jesus', 'Delight', 'Disciples Chosen', 'Doll Face',
       'God&#39;s Rescue Plan', 'Great Commission and Ascension',
       'Healing of the Demoniac', 'Invisible', 'JESUS',
       'Jairus&#39;s Daughter Brought Back to Life',
       'Jesus Calms the Storm', 'Jesus Carries His Cross',
       'Jesus Feeds 5,000', 'Jesus Predicts His Death and Resurrection',
       'Jesus Proclaims Fulfillment of the Scriptures',
       'Jesus Spends Time with Sinners', 'Jesus is Betrayed and Arrested',
       'Jesus is Brought to Herod', 'Jesus is Crucified',
       'Jesus is Mocked and Questioned', 'Jesus is Sentenced',
       

In [15]:
df_90.geo__country.value_counts()

India         937
Egypt         287
Bangladesh    207
China         196
Name: geo__country, dtype: int64

In [16]:
country_groupby = df.groupby('geo__country')

In [17]:
country_groupby['video_title'].unique()

geo__country
Bangladesh    [#FallingPlates, 1. Jesus, Our Loving Pursuer,...
China         [#FallingPlates, #掉落的盘子, 40天与耶稣同在, 7天与耶稣同在：恩典,...
Egypt         [#FallingPlates, 1- في العائلة, 2- عمل يدوي, 2...
India         [#FallingPlates, #फालिंगप्लेट्स , 1. Jesus, Ou...
Name: video_title, dtype: object

In [18]:
india = country_groupby.get_group('India')

In [19]:
falling_plates_india = india[india['video_title'] == '#FallingPlates']

In [20]:
event_score = falling_plates_india.event_name.value_counts()

In [21]:
event_score.index

Index(['videostarts', 'videoplay', 'a_media_progress10', 'a_media_progress25',
       'a_media_progress50', 'a_media_progress90', 'a_media_progress75'],
      dtype='object')

In [22]:
keys = df.video_title.unique()
reviews = dict.fromkeys(keys, 0)

In [23]:
for row in df.columns:
    print(row)

geo__country
event_name
video_title


In [24]:
#tried to make a function by iterating through a groupby. couldn't figure it out.

# def rating_calculator(group):
#     rating = 0
#     reviews = dict.fromkeys(set(group.video_title), 0)
#     for event_name in group.event_name:
#         if event_name == 'videostarts' or event_name == 'videoplay':
#             reviews[group.video_title] += 1
#         elif event_name == 'a_media_progress10':
#             reviews[group.video_title] += (1 * .1)
#         elif event_name == 'a_media_progress25':
#             reviews[group.video_title] += (1 * .25)
#         elif event_name == 'a_media_progress50':
#             reviews[group.video_title] += (1 * .5)
#         elif event_name == 'a_media_progress75':
#             reviews[group.video_title] += (1 * .75)
#         elif event_name == 'a_media_progress90':
#             reviews[group.video_title] += (1 * .9)
#     print(f'total: {rating}\nnormalized: {rating/len(events)}')
#     return (reviews)
# #     return
# # print(rating)
# # print(rating/len(event_score))

In [25]:
#rating_calculator(event_score)

In [26]:
def review_builder(country_groupby):
    review_dic = dict.fromkeys(country_groupby.groups.keys(), None)
    print(review_dic)
    for country, group in country_groupby:
        review_dic[country] = rating_calculator(group)
    return review_dic

In [27]:
(29 + 28 + (22*.1) + (21*.25) + (20*.5) + (13*.9) + (13*.75))

95.9

In [28]:
def view_to_rating(event_name):
    d ={
        'videostarts':1,
        'videoplay':1, 
        'a_media_progress10':(1*.1),
        'a_media_progress25':(1*.25),
        'a_media_progress50':(1*.5), 
        'a_media_progress75':(1*.75), 
        'a_media_progress90':(1*.9)
    }
    
    return(d[event_name])

In [29]:
view_to_rating('a_media_progress90')

0.9

In [30]:
df['score'] = df.event_name.apply(view_to_rating)

In [77]:
groupby = df.groupby(['geo__country','video_title'])

In [78]:
review_scores = groupby.sum()

In [92]:
len(review_scores.values)

612

In [80]:
len(review_scores.index)

612

In [73]:
from scipy import sparse

In [74]:
data = [3, 9, 5]
rows = [0, 1, 1]
cols = [2, 1, 2]

sparse_matrix = sparse.csr_matrix((data, (rows, cols)), 
                                  shape=(len(rows), len(cols)))
sparse_matrix.toarray()

array([[0, 0, 3],
       [0, 9, 5],
       [0, 0, 0]])

In [81]:
sparse.csr_matrix(review_scores.values)

<612x1 sparse matrix of type '<class 'numpy.float64'>'
	with 612 stored elements in Compressed Sparse Row format>

In [95]:
my_lil_sparse_matrix = df.pivot_table(columns='video_title', index='geo__country', aggfunc='sum', fill_value=0)
#df_final.fillna(0, inplace = True)

In [96]:
my_lil_sparse_matrix

score                                       \
video_title  #FallingPlates #फालिंगप्लेट्स  #掉落的盘子 1- في العائلة   
geo__country                                                       
Bangladesh            11.35               0    0.0           0.0   
China                  9.00               0   13.5           0.0   
Egypt                 15.35               0    0.0           2.1   
India                 95.90               1    0.0           0.0   

                                                                       \
video_title  1. Jesus, Our Loving Pursuer 1.가족 중에 1.우리의 사랑의 추구자이신 예수님   
geo__country                                                            
Bangladesh                            4.5       0                   0   
China                                 0.0       0                   0   
Egypt                                 0.0       0                   0   
India                                30.0       2                   1   

                                                          \
video_title  2- عمل يدوي 2. Jesus, Our Gracious Forgiver   
geo__country                                               
Bangladesh           0.0                             3.6   
China                0.0                             0.0   
Egypt                2.1                             4.4   
India                0.0                            12.6   

                                             ...                            \
video_title  3. Jesus, Our Power for Living  ... 路加福音简介 身份错认于路司得 返回叙利亚的安提阿   
geo__country                                 ...                             
Bangladesh                              0.0  ...   0.00      0.0       0.0   
China                                   0.0  ...   6.85      4.5       4.5   
Egypt                                   0.0  ...   0.00      0.0       0.0   
India                                   4.5  ...   0.00      0.0       0.0   

                                                                             
video_title  选出马提亚替代犹大 那些使人犯罪的人有祸了 钉在十字架上的囚犯 雅典的偶像 非斯都咨询亚基帕王 高贵的比里亚人 魔鬼引诱耶稣  
geo__country                                                                 
Bangladesh         0.0         0.0      0.00   0.0       0.0     0.0   0.00  
China              4.5         3.6      6.45   4.5       5.5     4.5   2.85  
Egypt              0.0         0.0      0.00   0.0       0.0     0.0   0.00  
India              0.0         0.0      0.00   0.0       0.0     0.0   0.00  

[4 rows x 406 columns]

In [38]:
country_groupby.size()

geo__country
Bangladesh    1953
China         1936
Egypt         2467
India         8644
dtype: int64

In [39]:
country_groupby.describe()

score                                               
               count      mean       std  min   25%   50%  75%  max
geo__country                                                       
Bangladesh    1953.0  0.645008  0.352061  0.1  0.25  0.75  1.0  1.0
China         1936.0  0.666942  0.347348  0.1  0.25  0.75  1.0  1.0
Egypt         2467.0  0.652047  0.350626  0.1  0.25  0.75  1.0  1.0
India         8644.0  0.661071  0.349209  0.1  0.25  0.75  1.0  1.0

In [40]:
index = df.geo__country.unique()

In [41]:
columns = df.video_title.unique()

In [42]:
data = np.zeros((index.shape[0], columns.shape[0]))

In [43]:
df.event_name.value_counts().index

Index(['videoplay', 'videostarts', 'a_media_progress10', 'a_media_progress25',
       'a_media_progress50', 'a_media_progress75', 'a_media_progress90'],
      dtype='object')

In [44]:
reviews = pd.DataFrame(data, index=index, columns=columns)

In [45]:
reviews

,#FallingPlates,"1. Jesus, Our Loving Pursuer","2. Jesus, Our Gracious Forgiver","6. Jesus, Our Complete Restorer",7. Jesus Our Living Water,Abraham,Angels at the Tomb,Annas Questions Jesus&#39;s Authority,Announcement to Mary,B Me,...,Fracture,Getting Started Is Easy,God&#39;s Word Becomes Flesh,Good,Great Confrontation,Grief will Turn to Joy!,Growing In Christ,Handiwork,Idols in Athens,In Time
Bangladesh,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
China,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Egypt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
India,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [46]:
df.head()

,geo__country,event_name,video_title,score
0,Bangladesh,videoplay,#FallingPlates,1.00
1,Bangladesh,a_media_progress10,#FallingPlates,0.10
2,Bangladesh,a_media_progress90,#FallingPlates,0.90
3,Bangladesh,videostarts,#FallingPlates,1.00
4,Bangladesh,a_media_progress75,#FallingPlates,0.75
